In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Сделаем базовый код для обучения персептрона с помощью стохастического градиентного спуска (по одной точке)

In [74]:
def score(weights, features, bias):
    return np.dot(weights, features) + bias

def step(x):
    return 1 if x >= 0 else 0

def predict(weights, features, bias):
    return step(score(weights, features, bias))

def mean_perceptron_error(weights, features, bias, labels):
    errors = [0 if predict(weights, features[i], bias) == labels[i] else np.abs(score(weights, features[i], bias)) for i in range(len(features))]
    return np.sum(errors) / len(features)

In [75]:
def stohastic_gradient(weights, features, bias, labels, learning_rate=0.01):
    predctions = predict(weights, features, bias)
    weights += learning_rate * (labels - predctions) * features
    bias += learning_rate * (labels - predctions)
    return weights, bias


In [76]:
def perceptron(features, labels, learning_rate=0.01, epochs=500):
    weights = np.ones(features.shape[1])
    bias = 0
    for _ in range(epochs):
        dot = np.random.randint(0, features.shape[0])
        weights, bias = stohastic_gradient(weights, features[dot], bias, labels[dot], learning_rate)
    return weights, bias

In [77]:
class Perceptron:
    def __init__(self, learning_rate=0.01, epochs=500):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def fit(self, features, labels):
        self.weights, self.bias = perceptron(features, labels, self.learning_rate, self.epochs)

    def predict(self, features):
        return np.array([predict(self.weights, feature, self.bias) for feature in features])
    
    def mean_error(self, features, labels):
        if self.weights is None or self.bias is None:
            print("Модель ещё не обучена")
            return None
        return mean_perceptron_error(self.weights, features, self.bias, labels)

## Обучим персептрон на данных из книги

### Упражнение 5.1

In [78]:
df = pd.DataFrame({"кашель":[0, 1, 1, 1, 1, 0, 0, 0],
                   "температура":[1, 1,0, 1, 0, 1, 1, 0],
                   "затрудненное дыхание":[1, 0, 1, 1, 0, 1, 0, 0],
                   "утомляемость":[1, 1, 1, 0, 1, 0, 0, 1],
                   "диагноз":[1, 1, 1, 1, 0, 0, 0, 0]}, index=np.arange(1, 9))
df.index.name = "Пациент"
df.head()

,кашель,температура,затрудненное дыхание,утомляемость,диагноз
Пациент,,,,,
1,0,1,1,1,1
2,1,1,0,1,1
3,1,0,1,1,1
4,1,1,1,0,1
5,1,0,0,1,0


In [79]:
labels = df["диагноз"]
features = df.drop(columns=["диагноз"])


In [80]:
model = Perceptron()
model.fit(features.values, labels.values)
predictions = model.predict(features.values)


In [81]:
print(f"Веса: {model.weights}, смещение: {model.bias}, ошибка {model.mean_error(features.values, labels.values)}")

Веса: [0.58 0.46 0.59 0.47], смещение: -1.0700000000000007, ошибка 0.0


### Упражнение 5.2

Создадим модель из книги и выполним с ней манипуляции

In [92]:
bad_model = Perceptron()
bad_model.weights = np.array([2, 3], dtype=float)
bad_model.bias = -4


In [86]:
#Точка p(1, 1) с меткой 0 должна классифицироваться неверно
p = np.array([1, 1])
bad_model.predict([p])


array([1])

In [84]:
# рассчитаем ошибку (среднюю) для точки
bad_model.mean_error([p], [0])

np.float64(1.0)

In [95]:
# сделаем один шаг градиентного спуска
prediction = bad_model.predict([p])
bad_model.weights += bad_model.learning_rate * (0 - prediction) * p
bad_model.bias += bad_model.learning_rate * (0 - prediction)
# проверим, что вес изменился
print(bad_model.weights, bad_model.bias)

[1.97 2.97] [-4.02]


In [96]:
bad_model.mean_error([p], [0])# ошибка должна уменьшиться

np.float64(0.9200000000000008)

### Упражнение 5.3

Создадим персептрона модулирующий AND, OR, XOR
(спойлер AND = x1 + x2 - 2
OR = x1 + x2, XOR не существует)

In [103]:
data = pd.DataFrame({"x1":[0, 0, 1, 1],
                   "x2":[0, 1, 0, 1],
                   "y":[0, 0, 0, 1]})


In [102]:
AND = Perceptron()
AND.fit(data[["x1", "x2"]].values, data["y"].values)
predictions = AND.predict(data[["x1", "x2"]].values)
predictions

array([0, 0, 0, 1])

In [104]:
data_or = pd.DataFrame({"x1":[0, 0, 1, 1],
                   "x2":[0, 1, 0, 1],
                   "y":[0, 1, 1, 1]})

In [106]:
OR = Perceptron()
OR.fit(data_or[["x1", "x2"]].values, data_or["y"].values)
predictions = OR.predict(data_or[["x1", "x2"]].values)
predictions

array([0, 1, 1, 1])